In [3]:
from googletrans import Translator
import pandas as pd
import numpy as np
import time
import re
import pickle

In [6]:
df_kospi_filing_2011_2015 = pd.read_excel('C:\\Users\\sylim2357\\Documents\\SNU\\FVID\\10k delta\\Data\\kospi200_2011_2015.xlsx')

In [21]:
missing_list = [7,12,13,14,19,20,27,56,99,100,101,102,103,104,105,106,108,109,111,112,114,115,117,119,120,121,122,127,131,136,137,139,141,173,198,199,203,206,208,211,212,218,220,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,369,370,372,373,374,375]

In [84]:
df_kospi_filing_2011_2015['Text'].replace("\\n|전자공시시스템 dart\.fss\.or\.kr(\nPage \d{1,2})?(?=\s)|\s+", " ", regex=True)[0]

'II. 사업의 내용  1. 사업의 개요 국내 면방직산업은 해방 이후 우리나라의 산업 발전을 주도해 온 산업으로 오랜 역사를 가지고 있으며 노동 집약 산업으로 과거 70~80년대를 거쳐 값싼 노동력을 활용한 많은 기업들이 신규로 면방 산업에 진입하여 설비규모가 확대되고 최고의 전성기를 이루었습니다. 그러나 90년대 들어서면서 가중되는 인력난과 저개발 국가들로부터 수입되는 값싼 제품들로 인하여 경쟁력이 약화되어 많은 기업이 도태되었으며 최근에는 기업의 구조조정과 제품 전문화에 성공한 몇몇 기업을 중심으로 새로운 발전을 모색하고 있습니다. 면방직 업계의 매출은 섬유산업의 기초 원자재를 공급하는 형태로 내수의 경우 국내 경기부진에 따라 소비가 감소하고 수출은 선진국 경기회복이 지연됨에 따라 수출오더가 감소하여 재고증가와 수익성 악화로 전반적으로 영업환경이 어려운 상황입니다. 또한 면사의 주원료인 원면은 100%로 수입에 의존하기 때문에 환율 이나 경기변동 및 작황에 따라 가격이 민감하게 움직이고 제조원가에서 큰 비중을 차지하므로 원가절감에도 한계가 있어 채산성을 확보하는 데 다소 어려움이 있습 니다. 이러한 환경하에서 국내 면방직 업계는 개발도상국과의 중복생산을 피하고 선진국형 섬유산업으로 탈바꿈하고자 고부가가치 신제품 개발과 특화된 제품 생산으로 수익성 확보 및 차별화를 이루고 노후화된 설비의 보완 및 자동화를 지속적으로 추진하여 생산성 및 품질면에서도 개발도상국들보다 시장우위를 점할 수 있도록 노력하고 있습니다. 2. 회사의 현황 1) 영업개황  지난 2011년도 상반기 한국경제는 1분기까지 연 5%이상의 양호한 경제회복세 를 유지하였으나 2분기 들어 세계경기회복에 대한 불확실성 증가와 더불어 경제 성장률이 둔화되면서 상반기 전체로 3.7% 정도의 성장세를 보일 것으로 전망됩 니다. 또한 2011년도 2분기 들어서 다시 불거진 유럽의 재정위기와 일본의 대지진 사태, 달러약세를 촉발한 미국의 부채한도 증액협상 난항으로 인하여 2011년 1분기까지 평균 30%대의 

In [ ]:
def make_sublist(text):
    batch = []
    batch_size = 5
    text_list = [text for text in re.split('\. |Page', text) if (len(text)>3) & ('dart.fss.or.kr' not in text)]
    num_batches = int(np.ceil(len(text_list)/batch_size))
    for i in range(num_batches):
        batch.append('. '.join(text_list[batch_size*i:batch_size*(i+1)]))
    return batch

text_series = df_kospi_filing_2011_2015['Text'].replace("\s\s+|\\n", " ", regex=True)
text_series_sublists = text_series.apply(make_sublist)
trans_error = []
translated_series = []
for idx, text_sublist in enumerate(text_series_sublists):
    try:
        time.sleep(3)
        print(str(idx)+' being translated')
        translated = Translator().translate(text_sublist)
        print(str(idx)+' has been translated')
        translated_series.append('. '.join([trans.text for trans in translated]))
        print(str(idx)+' has been recorded')
    except:
        try:
            text_sublist_sublist = []
            print(str(idx)+' being translated again')
            for i in range(int(np.ceil(len(text_sublist)/batch_size))):
                time.sleep(3)
                translated = Translator().translate(text_sublist[batch_size*i:batch_size*(i+1)])
                text_sublist_sublist.append('. '.join([trans.text for trans in translated]))
            print(str(idx)+' has been translated')
            translated_series.append('. '.join(text_sublist_sublist))
            print(str(idx)+' has been recorded')
        except:
            print(str(idx)+' has error')
            trans_error.append(idx)
            translated_series.append(np.nan)
    if idx % 100 == 0:
            with open('translated.pickle', 'wb') as f:
                pickle.dump(translated_series, f)
            with open('trans_error.pickle', 'wb') as f:
                pickle.dump(trans_error, f)

0 being translated
0 has error
1 being translated
1 has error
2 being translated
2 has error


In [55]:
re.split('\. |Page', df_kospi_filing_2011_2015.iloc[7].replace("\s\s+|\\n", " ", regex=True).Text)

['II',
 '사업의 내용 1',
 '사업의 개요 가',
 '사업부문별 현황 당사는 식품, 식품소비재, AM, 산업자재, 용기, 사료, 무역, 의약 등 총 8개의 BU(Business Unit)로 구성되어 있습니다',
 '당사를 제외하고 총 17개의 계열회사가 있는데 연결대상인 14개의 종속회사와 지분법적용 대상인 3개의 관계회사(삼남석유화학, 삼양화성, 비엔엠유통 )가 있습니다',
 '상기한 각 BU와 종속회사, 관계회사에 걸쳐서 크게 식품부문, 화학부문, 사료 부문, 무역부문, 기타부문의 사업을 영위하고 있습니다',
 '당사 및 연결대상인 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다',
 '사업부문 회 사 주요재화 및 용역 식품부문 삼양사 삼양제넥스 삼양밀맥스 삼양웰푸드 진황도삼양제넥스식품유한공사 세븐스프링스 삼양푸드앤다이닝 삼양사 화학부문 삼양공정소료(상해)유한공사 삼양EP헝가리 삼양이노켐 삼양이엠에스 사료부문 삼양사 무역부문 삼양사 기타부문 삼양사 삼양제넥스 삼양엔텍 삼양데이타시스템 삼양리서치(SRC) PT',
 'Samyang IDB 설탕, 고감미도감미료, 올리브유 등 전분, 전분당, 당알코올, 기능성식품 밀가루, 프리믹스 마가린, 쇼트닝, 식용유 전분당 패밀리레스토랑 베이커리 카페 엔지니어링플라스틱,PET병, PET병 재활용 이온교환수지, 산업자재용 섬유 엔지니어링플라스틱 엔지니어링플라스틱 BPA 컬럼스페이서, 유기절연막 배합사료 수출입 패취제, 의료용구, 항암제 항암제 제지기계, 산업기계, 환경설비 정보통신 의약품연구 카사바 경작 등 1)식품부문 가) 산업의 특성 대부분의 원료를 해외에서 수입하여 가공 후 판매하기 때문에 국제 곡물가의 영향을 많이 받고 있습니다',
 '원당, 옥수수, 원맥 등을 수입해서 1차 가공하고 있는데 원당은 설탕, 옥수수 는 사료와 전분당, 원맥은 밀가루의 원료로 주로 쓰입니다',
 '제당업계는 1950년대 말 삼양사, CJ제일제당, 대한제당의 3사 체제로 정비된 이후신규진 입 사업자 없이 현재에 이르고 있으며,

In [68]:
def make_sublist(text):
    batch = []
    batch_size = 5
    text_list = [text for text in re.split('\. |Page', text) if (len(text)>3) & ('dart.fss.or.kr' not in text)]
    num_batches = int(np.ceil(len(text_list)/batch_size))
    for i in range(num_batches):
        batch.append('. '.join(text_list[batch_size*i:batch_size*(i+1)]))
    return batch
text_sublist = make_sublist(df_kospi_filing_2011_2015.iloc[7].replace("\s\s+|\\n", " ", regex=True).Text)

['당사 및 연결대상인 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다', '사업부문 회 사 주요재화 및 용역 식품부문 삼양사 삼양제넥스 삼양밀맥스 삼양웰푸드 진황도삼양제넥스식품유한공사 세븐스프링스 삼양푸드앤다이닝 삼양사 화학부문 삼양공정소료(상해)유한공사 삼양EP헝가리 삼양이노켐 삼양이엠에스 사료부문 삼양사 무역부문 삼양사 기타부문 삼양사 삼양제넥스 삼양엔텍 삼양데이타시스템 삼양리서치(SRC) PT', 'Samyang IDB 설탕, 고감미도감미료, 올리브유 등 전분, 전분당, 당알코올, 기능성식품 밀가루, 프리믹스 마가린, 쇼트닝, 식용유 전분당 패밀리레스토랑 베이커리 카페 엔지니어링플라스틱,PET병, PET병 재활용 이온교환수지, 산업자재용 섬유 엔지니어링플라스틱 엔지니어링플라스틱 BPA 컬럼스페이서, 유기절연막 배합사료 수출입 패취제, 의료용구, 항암제 항암제 제지기계, 산업기계, 환경설비 정보통신 의약품연구 카사바 경작 등 1)식품부문 가) 산업의 특성 대부분의 원료를 해외에서 수입하여 가공 후 판매하기 때문에 국제 곡물가의 영향을 많이 받고 있습니다', '원당, 옥수수, 원맥 등을 수입해서 1차 가공하고 있는데 원당은 설탕, 옥수수 는 사료와 전분당, 원맥은 밀가루의 원료로 주로 쓰입니다', '제당업계는 1950년대 말 삼양사, CJ제일제당, 대한제당의 3사 체제로 정비된 이후신규진 입 사업자 없이 현재에 이르고 있으며, 이는 제당산업이 장치산업이며 저성장산업이기 때문 입니다']


In [25]:
len(make_sublist(df_kospi_filing_2011_2015['Text'].replace("\s\s+|\\n", " ", regex=True).iloc[7]))

39

In [69]:
text_sublist = make_sublist(df_kospi_filing_2011_2015['Text'].replace("\s\s+|\\n", " ", regex=True).iloc[7])
translated_series = []
text_sublist_sublist = []
batch_size = 3
for i in range(int(np.ceil(len(text_sublist)/batch_size))):
    time.sleep(3)
    print(i)
    translated = Translator().translate(text_sublist[batch_size*i:batch_size*(i+1)])
    text_sublist_sublist.append('. '.join([trans.text for trans in translated]))
translated_series.append('. '.join(text_sublist_sublist))

['당사 및 연결대상인 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다', '사업부문 회 사 주요재화 및 용역 식품부문 삼양사 삼양제넥스 삼양밀맥스 삼양웰푸드 진황도삼양제넥스식품유한공사 세븐스프링스 삼양푸드앤다이닝 삼양사 화학부문 삼양공정소료(상해)유한공사 삼양EP헝가리 삼양이노켐 삼양이엠에스 사료부문 삼양사 무역부문 삼양사 기타부문 삼양사 삼양제넥스 삼양엔텍 삼양데이타시스템 삼양리서치(SRC) PT', 'Samyang IDB 설탕, 고감미도감미료, 올리브유 등 전분, 전분당, 당알코올, 기능성식품 밀가루, 프리믹스 마가린, 쇼트닝, 식용유 전분당 패밀리레스토랑 베이커리 카페 엔지니어링플라스틱,PET병, PET병 재활용 이온교환수지, 산업자재용 섬유 엔지니어링플라스틱 엔지니어링플라스틱 BPA 컬럼스페이서, 유기절연막 배합사료 수출입 패취제, 의료용구, 항암제 항암제 제지기계, 산업기계, 환경설비 정보통신 의약품연구 카사바 경작 등 1)식품부문 가) 산업의 특성 대부분의 원료를 해외에서 수입하여 가공 후 판매하기 때문에 국제 곡물가의 영향을 많이 받고 있습니다', '원당, 옥수수, 원맥 등을 수입해서 1차 가공하고 있는데 원당은 설탕, 옥수수 는 사료와 전분당, 원맥은 밀가루의 원료로 주로 쓰입니다', '제당업계는 1950년대 말 삼양사, CJ제일제당, 대한제당의 3사 체제로 정비된 이후신규진 입 사업자 없이 현재에 이르고 있으며, 이는 제당산업이 장치산업이며 저성장산업이기 때문 입니다']
0
1
2
3
4
5
6
7
8
9
10
11
12


In [71]:
len(translated_series)

1